# ABT default

## Instalación de paquetes

In [2]:
pip install awswrangler

  Using cached awswrangler-3.9.1-py3-none-any.whl.metadata (17 kB)
Using cached awswrangler-3.9.1-py3-none-any.whl (381 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import sys
!{sys.executable} -m pip install PyAthena

  Using cached pyathena-3.9.0-py3-none-any.whl.metadata (6.3 kB)
Using cached pyathena-3.9.0-py3-none-any.whl (75 kB)


In [4]:
import awswrangler as wr
import pandas as pd
import datetime as dt
from datetime import datetime
from pyathena import connect
import matplotlib.pyplot as plt
import seaborn as sns

## Query

In [5]:
# DB Setting
bucket_name = 's3://viamericas-datalake-dev-us-east-1-283731589572-athena/'

#Connection
conn = connect(s3_staging_dir=f'{bucket_name}queryresults/',
               region_name='us-east-1')

df = pd.read_sql('''with viacheck as(
SELECT
    DATE_FORMAT(ct.CheckDate, '%Y-%m-%d') AS DATE,
    COUNT(DISTINCT ct.checkID) AS TX_check,
    SUM(ct.AMOUNT) AS TOTAL_AMOUNT_check,
    b.id_location
FROM
    AwsDataCatalog.viamericas.checkTable ct
LEFT JOIN
    AwsDataCatalog.viamericas.BRANCH a ON a.id_branch = ct.CheckBranch
LEFT JOIN
    AwsDataCatalog.viamericas.location b ON a.id_location = b.id_location
       
WHERE
   NOT (a.id_branch LIKE 'T%')
   AND a.id_flag_branch = 'A'
   AND DATE_FORMAT(ct.CheckDate, '%Y-%m') between '2023-01' and '2024-02'
   AND  NOT b.id_location = 'NULL'
GROUP BY
    DATE_FORMAT(ct.CheckDate, '%Y-%m-%d'),
    b.id_location
    --b.name_location;
)
SELECT
   CAST(RTRIM(b.ID_LOCATION) AS VARCHAR(60)) AS AGENCY,
   d.ID_GROUP_COMPANY AS ID_GROUP_COMPANY,
   a.ID_BRANCH as ID_BRANCH,
   CAST(DATE_FORMAT(a.DATE_RECEIVER, '%Y-%m-%d') AS DATE) AS DATE,
   SUM(CASE WHEN (A.ID_FLAG_RECEIVER = 'A' OR A.ID_FLAG_RECEIVER = 'C') THEN 0 ELSE 1 END) AS TX,
   SUM(CASE WHEN (A.ID_FLAG_RECEIVER = 'A' OR A.ID_FLAG_RECEIVER = 'C') THEN 0 ELSE A.NET_AMOUNT_RECEIVER END) as AMOUNT,
   c.BALANCE as BALANCE,
   coalesce(d.TX_check, 0) as TX_check,
   coalesce(d.TOTAL_AMOUNT_check, 0) as TOTAL_AMOUNT_check,
   c.dso_deposit_day as DSO_DEPO_DAY,
   CAST(DATE_FORMAT(d.installation_date, '%Y-%m-%d') AS DATE) as START_DATE
 FROM
   AwsDataCatalog.viamericas.RECEIVER a
   INNER JOIN AwsDataCatalog.viamericas.GROUP_BRANCH p ON p.ID_MAIN_BRANCH = CASE WHEN a.ID_MAIN_BRANCH_EXPIRED IS NULL THEN RTRIM(a.ID_MAIN_BRANCH_SENT) ELSE RTRIM(a.ID_MAIN_BRANCH_EXPIRED) END
   LEFT JOIN AwsDataCatalog.viamericas.BRANCH b ON a.ID_BRANCH = b.ID_BRANCH
   LEFT JOIN AwsDataCatalog.viamericas.history_balance c ON c.id_main_branch = b.id_main_branch and c.date_balance = cast(a.DATE_RECEIVER as DATE)
   LEFT JOIN viacheck d ON d.id_location = b.ID_LOCATION and cast(d.DATE as DATE) = cast(a.DATE_RECEIVER as DATE)
   left join AwsDataCatalog.viamericas.locations d on b.ID_LOCATION = d.id
   
 WHERE
   NOT (A.ID_MAIN_BRANCH_SENT LIKE 'M%')
   AND NOT (A.ID_BRANCH LIKE 'T%')
   AND SUBSTRING(b.id_branch, 1, 1) IN (SELECT SUBSTRING(PREFIX, 1, 1) FROM AwsDataCatalog.viamericas.BRANCH_PREFIX P WHERE TRIM(B.ID_COUNTRY) = TRIM(P.ID_COUNTRY))
   AND A.DATE_RECEIVER >= TIMESTAMP '2023-01-01 00:00:00.000'
   AND A.DATE_RECEIVER < TIMESTAMP '2024-02-04 00:00:00.000'
   AND b.ID_LOCATION IS NOT NULL
   AND b.ID_LOCATION NOT IN ('MD0010', 'MD0952', 'AK0003', 'CA3897', 'NY1130', 'MD0696', 'FL1933', 'AK0004', 'CA4046', 'NY1221', 'MD0623', 'MD1003', 'MD1018', 'AK0008', 'CA4291', 'NY1346', 'CA4350', 'NY1381','OK0236','FL2287','CA4391','AK0009', 'NY1397','CA4392','AK0010','FL2288','NY1399','AK0012','CA4396','NY1402','FL2289','AK0013', 'CA4418','NY1410','FL2301','AK0014', 'CA4428', 'NY1413')
   AND a.NET_AMOUNT_RECEIVER <> 0
 GROUP BY
   RTRIM(b.ID_LOCATION),
   d.ID_GROUP_COMPANY,
   a.ID_BRANCH,
   c.BALANCE, 
   d.TX_check,
   d.TOTAL_AMOUNT_check,
   c.dso_deposit_day,
   d.installation_date,
   CAST(DATE_FORMAT(a.DATE_RECEIVER, '%Y-%m-%d') AS DATE); ''', conn)

/tmp/ipykernel_21/975165692.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('''with viacheck as(


In [6]:
df.head()

,AGENCY,ID_GROUP_COMPANY,ID_BRANCH,DATE,TX,AMOUNT,BALANCE,TX_check,TOTAL_AMOUNT_check,DSO_DEPO_DAY,START_DATE
0,CA4507,USA011899,A24704,2023-10-30,1,90.00,9800.0000,0,0.0,0.0,2023-06-20
1,TX1320,USA019463,A23417,2023-05-20,4,990.00,3456.5500,0,0.0,NaN,2022-10-25
2,CA3461,USA016308,A18631,2023-05-13,17,9393.15,5635.0767,0,0.0,NaN,2019-11-07
3,WA0244,USA016298,A18616,2023-06-23,1,250.00,3518.1303,0,0.0,0.0,2019-11-22
4,NJ1155,USA020676,A25231,2023-12-11,21,8428.00,13748.2492,0,0.0,3.0,2019-12-18


### Query money orders (MO)

In [8]:
# DB Setting
bucket_name = 's3://viamericas-datalake-dev-us-east-1-283731589572-athena/'

#Connection
conn = connect(s3_staging_dir=f'{bucket_name}queryresults/',
               region_name='us-east-1')

MO = pd.read_sql('''SELECT
   CAST(DATE_FORMAT(a.TRANSACTION_DATE, '%Y-%m-%d') AS DATE) AS DATE,
   COUNT(DISTINCT a.ID_MONEYORDERS_SALES) AS TX_MO,
   SUM(a.AMOUNT) AS TOTAL_AMOUNT_MO,
   r.id_branch as ID_BRANCH
FROM
   AwsDataCatalog.viamericas.vcw_moneyorders_sales a
   LEFT JOIN AwsDataCatalog.viamericas.branch b ON b.ID_BRANCH = a.id_branch
   LEFT JOIN AwsDataCatalog.viamericas.branch_agent f ON f.id_branch =b.id_branch
   LEFT JOIN AwsDataCatalog.viamericas.chain g ON g.id_chain =b.id_chain
   LEFT JOIN AwsDataCatalog.viamericas.BUDGET_LOCATIONS_INCLUDED h ON h.id_location =b.id_location
   LEFT JOIN AwsDataCatalog.analytics.location_branch_reference r ON r.id_branch = b.id_branch
WHERE
   NOT (b.ID_MAIN_BRANCH LIKE 'M%')
   AND NOT (a.id_branch LIKE 'T%')
   AND b.id_flag_branch = 'A'
   AND DATE_FORMAT(TRANSACTION_DATE, '%Y-%m') > '2022-12' AND DATE_FORMAT(TRANSACTION_DATE, '%Y-%m') < '2024-02' 
   AND  NOT r.id_location = 'NULL'
   AND b.name_branch not like '%DUMMY%'
 GROUP BY
   CAST(DATE_FORMAT(a.TRANSACTION_DATE, '%Y-%m-%d') AS DATE),
   r.id_branch
  ORDER BY 
   DATE;''', conn)

/tmp/ipykernel_21/3879106591.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  MO = pd.read_sql('''SELECT


In [12]:
# formateando las fechas
MO['DATE'] = pd.to_datetime(MO['DATE'])
df['DATE'] = pd.to_datetime(df['DATE'])

In [13]:
MO['DATE'] = MO['DATE'].dt.strftime('%Y-%m-%d')
df['DATE'] = df['DATE'].dt.strftime('%Y-%m-%d')

In [15]:
#Unir las MO
merged_df2 = pd.merge(df, MO, on=['DATE', 'ID_BRANCH'], how='outer')

In [16]:
# eliminar duplicados
merged_df2.drop_duplicates(subset=['DATE', 'AGENCY'], inplace = True)

### Query defaults

In [ ]:
# DB Setting
bucket_name = 's3://viamericas-datalake-dev-us-east-1-283731589572-athena/'

#Connection
conn = connect(s3_staging_dir=f'{bucket_name}queryresults/',
               region_name='us-east-1')

default = pd.read_sql('''WITH Payment_Agreeement as(
----------------------------------------------------------------------------------------------
--Agencias que han realizado un acuerdo de pago
----------------------------------------------------------------------------------------------
	SELECT
			DISTINCT
			REPLACE(RTRIM(b.ID_BRANCH),'C','A') AS ID_BRANCH --agencia que realizo el acuerdo
			,g.DATE_CREA_MAIN_BRANCH AS DATE_START -- Cuando empezo el acuerdo
			,g.DEBITO_GROUP AS  NEGOTIATED   --De cuanto fue el acuerdo
			,g.CREDITO_GROUP AS PAID -- Cuanto ha pagado la agencia del acuerdo
			,g.BALANCE_GROUP AS BALANCE --Banlance del acuerdo
	FROM	  AwsDataCatalog.viamericas.GROUP_BRANCH g INNER JOIN
			  AwsDataCatalog.viamericas.BRANCH b ON g.ID_MAIN_BRANCH = b.ID_MAIN_BRANCH
	WHERE	g.DEBITO_GROUP IS NOT NULL
			AND b.ID_LOCATION IS NOT NULL
			AND b.ID_BRANCH LIKE 'C%'
            AND g.DATE_CREA_MAIN_BRANCH >= '2022-01-01'
			AND g.DATE_CREA_MAIN_BRANCH < '2024-02-04'
), Audit_Events as(
	SELECT  
			a.DATE_AUDIT
			, b.ID_BRANCH
			, a.OLD_STATUS
			, a.OLD_FLAG
			, a.OLD_COMMENT
			, a.NEW_STATUS
			, a.NEW_FLAG
			, a.NEW_COMMENT
			, a.LAST_INACTIVATION_TYPE
			, i.DESC_INACTIVATION_TYPE
			, ROW_NUMBER() OVER(Partition By b.ID_BRANCH Order By b.ID_BRANCH, a.DATE_AUDIT ASC)As RowNum
	FROM	   AwsDataCatalog.viamericas.AUDIT_BRANCH_STATUS   a WITH   INNER JOIN
			   AwsDataCatalog.viamericas.INACTIVATION_TYPES   i WITH   ON i.ID_INACTIVATION_TYPE = a. LAST_INACTIVATION_TYPE   INNER JOIN
			Envio.  AwsDataCatalog.viamericas.BRANCH AS b WITH   ON a.ID_BRANCH = b.ID_BRANCH INNER JOIN
			 Envio  .   AwsDataCatalog.viamericas.GROUP_BRANCH g ON b.ID_MAIN_BRANCH = g.ID_MAIN_BRANCH
	WHERE	a. DATE_AUDIT   >= '2023-01-01'
			AND a. DATE_AUDIT   < '2024-02-04'
			AND (RTRIM(b.ID_BRANCH) LIKE 'A%')
			AND b.ID_TYPE_BRANCH = 'R'
            
			--AND b.ID_BRANCH IN('A00152','A00205','A00219','A01626','A01849')
			AND NOT (b.ID_BRANCH LIKE 'T%')
), PAST_DUE_AGENTS AS(
----------------------------------------------------------------------------------------------
--Agencias que han caido en PAST DUE de mas de 15 dias
----------------------------------------------------------------------------------------------
SELECT	
		DISTINCT i.ID_BRANCH
FROM	Audit_Events AS i LEFT JOIN
		Audit_Events AS a ON i.ID_BRANCH = a.ID_BRANCH AND i.DATE_AUDIT < a.DATE_AUDIT AND a.OLD_FLAG = 'I' AND a. NEW_FLAG   = 'A' AND i.RowNum = a.RowNum - 1 LEFT JOIN
		Payment_Agreeement p ON i.ID_BRANCH = p.ID_BRANCH AND CAST(i.DATE_AUDIT AS DATE) >= CAST(p.DATE_START AS DATE)
WHERE	i.OLD_FLAG = 'A'
		AND i. NEW_FLAG   = 'I'
		AND (  date_diff('day',i. DATE_AUDIT  , ISNULL(a.DATE_AUDIT, GETDATE())) >= 15 OR a.DATE_AUDIT IS NULL)
		AND i.LAST_INACTIVATION_TYPE IN(1,3,16)
		--AND i.DESC_INACTIVATION_TYPE IN('PAST DUE / BALANCE VENCIDO','BOUNCED / RETORNO','CHAIN PAST DUE / DEUDA EN CADENA'/*,'BUSINESS CLOSED / CIERRE DEL NEGOCIO'*/)
)
----------------------------------------------------------------------------------------------
--historial de inactivaciones de las agencias que han caido en PAST DUE de mas de 15 dias
----------------------------------------------------------------------------------------------
SELECT	
		i.DATE_AUDIT AS INACTIVATION_DATE
		, i.ID_BRANCH
		,    AwsDataCatalog.viamericas  . sfGetBranchStatus  (i.ID_BRANCH) AS CURRENT_EXTENDED_STATUS -- Status de la agencia a HOY
		, p.DATE_START AS  AGREEMENT_START_DATE   -- FECHA EN LA QUE EMPEZO EL ACUEROD DE PAGO
        ,  date_diff('day', p.DATE_START, i. DATE_AUDIT  ) AS  AGREEMENT_DAYS   -- CUANTO DIAS LLEVA EL ACUERDO A LA FECHA DEL EVENTO DE INACTIVACION DE ESTE REGISTRO
		, ISNULL(p.BALANCE, 0) AS  AGREEMENT_BALANCE   -- BALANCE A HOY DEL ACUERDO
		, ISNULL(p.NEGOTIATED, 0) AS  AGREEMENT_INITIAL_AMOUNT   -- VALOR INICIAL A HOY DEL ACUERDO
		, ISNULL(p.PAID, 0) AS  AGREEMENT_PAID   -- LO QUE SE A PAGADO A HOY DEL ACUERDO
		, i.OLD_STATUS
		, i.OLD_FLAG
		, i.OLD_COMMENT
		, i.NEW_STATUS
		, i.NEW_FLAG
		, i.NEW_COMMENT
		, i.LAST_INACTIVATION_TYPE
		, i.DESC_INACTIVATION_TYPE as DESC_LAST_INACTIVATION_TYPE
		, a.DATE_AUDIT as ACTIVATION_DATE
		,  date_diff('day',i. DATE_AUDIT  , ISNULL(a.DATE_AUDIT, GETDATE())) AS INACTIVE_DAYS
FROM	Audit_Events AS i INNER JOIN
		PAST_DUE_AGENTS pd ON i.ID_BRANCH = pd.ID_BRANCH LEFT JOIN
		Audit_Events AS a ON i.ID_BRANCH = a.ID_BRANCH AND i.DATE_AUDIT < a.DATE_AUDIT AND a.OLD_FLAG = 'I' AND a. NEW_FLAG   = 'A' AND i.RowNum = a.RowNum - 1 LEFT JOIN
		Payment_Agreeement p ON i.ID_BRANCH = p.ID_BRANCH AND CAST(i.DATE_AUDIT AS DATE) >= CAST(p.DATE_START AS DATE)
WHERE	i.OLD_FLAG = 'A'
		AND i. NEW_FLAG   = 'I'
ORDER BY i.ID_BRANCH, i.DATE_AUDIT ASC; ''', conn)
# correr en sql server los datos no están en athena (ni las funciones custom)

In [ ]:
# para muestra
#default = pd.read_csv('Datos Juan.csv')

In [ ]:
#quitando los espacios
default['ID_BRANCH'] = default['ID_BRANCH'].str.replace(' ','') 

In [ ]:
df3 = default[default['INACTIVE_DAYS'] > 6] #me quedo con 7 días inactivos en adelante

In [ ]:
# genero la variable de dfault
df3['DEFAULT'] = 1

In [ ]:
df3['DATE'] = pd.to_datetime(df3['INACTIVATION_DATE'])

In [ ]:
#filtro columnas
lista =['DATE', 'ID_BRANCH', 'DEFAULT']
df3 = df3[lista]

In [ ]:
# hay que darle el formato '%Y-%m-%d' a la fechas de default!
df3['DATE'] = df3['DATE'].dt.strftime('%Y-%m-%d')
merged_df2['DATE'] = pd.to_datetime(merged_df2['DATE'])
merged_df2['DATE'] = merged_df2['DATE'].dt.strftime('%Y-%m-%d')

In [ ]:
# agregado de casos de default
merged_df3 = pd.merge(merged_df2, df3, on=['DATE', 'ID_BRANCH'], how='outer')

In [ ]:
# descarto duplicados
merged_df3.drop_duplicates(subset=['DATE', 'ID_BRANCH'], inplace = True)

## Para generar variables que 'miran hacia atrás' hace falta tener para el período analizado todos los días para cada agencia

In [ ]:
#creo df básico para rellenar con fechas
lista =['DATE', 'ID_BRANCH']
stump = merged_df3[lista]

In [ ]:
# versión 3.0 para expandir lo mínimo
def fill_missing_dates(df):
    # Create an empty DataFrame with the specified date range
    date_range = pd.date_range(start=df['DATE'].min(), end=df['DATE'].max())
    df_fill = pd.DataFrame({'DATE': date_range})
    df_fill['DATE'] = pd.to_datetime(df_fill['DATE']).dt.date

    # Sort the original DataFrame by 'AGENCY' and 'DATE'
    df = df.sort_values(by=['ID_BRANCH', 'DATE'])

    # Create an empty DataFrame to hold the result
    result_df = pd.DataFrame()

    # Loop through each 'AGENCY'
    for agency in df['ID_BRANCH'].unique():
        df_aux = df[df['ID_BRANCH'] == agency]
        # Combining df_aux (AGENCY) with df_fill, keeping values from df_aux and filling dates, value1, value2 and value3
        merged_df = df_aux.set_index('DATE').combine_first(df_fill.set_index('DATE')).reset_index()
        
    # Fill missing values in 'other_column1', 'other_column2', ... columns
        columns_to_fill = ['ID_BRANCH']
        merged_df[columns_to_fill] = merged_df[columns_to_fill].ffill().bfill()

        result_df = pd.concat([result_df, merged_df], ignore_index=True)

    return result_df


In [ ]:
result_df = fill_missing_dates(stump)

In [ ]:
lista =['DATE', 'ID_BRANCH', 'TX', 'AMOUNT','BALANCE','TX_check','TOTAL_AMOUNT_check','TX_MO','TOTAL_AMOUNT_MO','DSO_DEPO_DAY','DEFAULT']
dinamic = merged_df3[lista]

In [ ]:
# agrego variables dinámicas al stumb con todas las fechas
merged_df4 = pd.merge(result_df, dinamic, on=['DATE', 'ID_BRANCH'], how='left')

In [ ]:
lista =['ID_BRANCH', 'AGENCY', 'ID_GROUP_COMPANY','START_DATE']
static = merged_df3[lista]

In [ ]:
static2 = static.drop_duplicates(subset=['ID_BRANCH'])

In [ ]:
# sumo las variables estáticas de cada agencia
merged_df5 = pd.merge(merged_df4, static2, on=['ID_BRANCH'], how='left')

In [ ]:
# posible punto para guardar lo alcanzado (si la memoria del kernel no lo permite ir uardando en pasos previos)
merged_df5.to_csv('default_ABT0.csv', index = False)

# Creación de la target

In [5]:
df = pd.read_csv('default_ABT0.csv')

In [6]:
df['DEFAULT'] = df['DEFAULT'].fillna(0)

In [7]:
# add day of week
df['DATE'] = pd.to_datetime(df['DATE'], format='mixed')
df['day_of_week'] = df['DATE'].dt.day_name()

In [8]:
# probar el duplicated para quedarse con los datos buenos
df['filtro'] = df.duplicated(subset=['DATE', 'ID_BRANCH'], keep='first')


In [ ]:
d8 = df[df['filtro'] == False]

In [ ]:
# generar una nueva variable para poder limpiar más fácil los duplicados
d8['DATE'] = d8['DATE'].dt.strftime('%Y-%m-%d')
d8['DATE1'] = d8['DATE'].astype(str)
d8['DATE_ID_BRANCH'] = d8['DATE1'] + '_' + d8['ID_BRANCH']

In [ ]:
d8['filtro2'] = d8.duplicated(subset=['DATE_ID_BRANCH'], keep='first')

In [ ]:
d9 = d8[d8['filtro2'] == False]

In [13]:
# forma de hacer la target
d9=d9.sort_values(by=['ID_BRANCH', 'DATE'])
d9['TARGET'] =d9.groupby('ID_BRANCH')['DEFAULT'].shift(-9)
# se adelanta el dato de default 9días, saber el lunes si el miercóles de la semana sieguiente la agencia quedará incativa por 7 o más días.

In [14]:
# borrar columnas no deseadas
lista = ['filtro', 'DATE1', 'DATE_ID_BRANCH', 'filtro2']
d9 = d9.drop(lista, axis=1)

In [15]:
# guardar
d9.to_csv('default_ABT1.csv', index = False)

# Generar variables que 'miran hacia atrás'

In [5]:
df = pd.read_csv('default_ABT1.csv')

In [6]:
df['DATE'] = pd.to_datetime(df['DATE'])

In [7]:
#sort values adecuado para todos los calculos hacia atrás!
df=df.sort_values(by=['ID_BRANCH', 'DATE'])

In [ ]:
def calculate_rolling_means(df, columns, window):
    # Create an empty DataFrame to hold the results
    result_df = pd.DataFrame()

    # Loop through each column in the list of columns
    for column in columns:
        # Calculate the rolling mean for the current column
        rolling_mean = df.set_index('DATE').groupby('ID_BRANCH', sort=False)[column].rolling(window).mean().round(2).to_numpy()

        # Add the rolling mean to the result DataFrame
        result_df[f'roll_mean_{window}_{column}'] = rolling_mean

    # Concatenate the original DataFrame with the result DataFrame
    result_df = pd.concat([df, result_df], axis=1)

    return result_df


In [10]:
df = calculate_rolling_means(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check'], 30)#'30D')
df = calculate_rolling_means(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check'], 7)#'7D')

In [ ]:
def calculate_rolling_max(df, columns, window):
    # Create an empty DataFrame to hold the results
    result_df = pd.DataFrame()

    # Loop through each column in the list of columns
    for column in columns:
        # Calculate the rolling mean for the current column
        rolling_mean = df.set_index('DATE').groupby('ID_BRANCH', sort=False)[column].rolling(window).max().to_numpy()

        # Add the rolling mean to the result DataFrame
        result_df[f'roll_max_{window}_{column}'] = rolling_mean

    # Concatenate the original DataFrame with the result DataFrame
    result_df = pd.concat([df, result_df], axis=1)

    return result_df

In [12]:
df = calculate_rolling_max(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check'],  7)#'7D')
df = calculate_rolling_max(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check'], 30)#'30D')

In [ ]:

def calculate_lag(df, columns, window):
    # Create an empty DataFrame to hold the results
    result_df = pd.DataFrame()

    # Loop through each column in the list of columns
    for column in columns:
        # Calculate the rolling mean for the current column
        rolling_mean = df.set_index('DATE').groupby('ID_BRANCH', sort=False)[column].shift(window).to_numpy()

        # Add the rolling mean to the result DataFrame
        result_df[f'lag_{window}_{column}'] = rolling_mean

    # Concatenate the original DataFrame with the result DataFrame
    result_df = pd.concat([df, result_df], axis=1)

    return result_df

In [14]:
df = calculate_lag(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check', 'DEFAULT'], 1)

In [16]:
df = calculate_lag(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check', 'DEFAULT'], 2)
df = calculate_lag(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check'], 7)
df = calculate_lag(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check', 'DEFAULT'], 30)

In [ ]:
def calculate_rolling_sum(df, columns, window):
    # Create an empty DataFrame to hold the results
    result_df = pd.DataFrame()

    # Loop through each column in the list of columns
    for column in columns:
        # Calculate the rolling mean for the current column
        rolling_mean = df.set_index('DATE').groupby('ID_BRANCH', sort=False)[column].rolling(window).sum().to_numpy()

        # Add the rolling mean to the result DataFrame
        result_df[f'roll_sum_{window}_{column}'] = rolling_mean

    # Concatenate the original DataFrame with the result DataFrame
    result_df = pd.concat([df, result_df], axis=1)

    return result_df

In [18]:
df = calculate_rolling_sum(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check'], 7)
df = calculate_rolling_sum(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check'], 30)
df = calculate_rolling_sum(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check'], 60)

In [19]:
df.columns

Index(['DATE', 'ID_BRANCH', 'TX', 'AMOUNT', 'BALANCE', 'TX_check',
       'TOTAL_AMOUNT_check', 'TX_MO', 'TOTAL_AMOUNT_MO', 'DSO_DEPO_DAY',
       ...
       'roll_sum_30_TX_check', 'roll_sum_30_DEFAULT', 'roll_sum_60_AMOUNT',
       'roll_sum_60_BALANCE', 'roll_sum_60_TOTAL_AMOUNT_MO',
       'roll_sum_60_TOTAL_AMOUNT_check', 'roll_sum_60_TX', 'roll_sum_60_TX_MO',
       'roll_sum_60_TX_check', 'roll_sum_60_DEFAULT'],
      dtype='object', length=105)

## Calcular antigüedad

In [26]:
df['DATE'] = pd.to_datetime(df['DATE'])
df['START_DATE'] = pd.to_datetime(df['START_DATE'])

In [27]:
df.dtypes

DATE                              datetime64[ns]
ID_BRANCH                                 object
TX                                       float64
AMOUNT                                   float64
BALANCE                                  float64
                                       ...      
roll_sum_60_TOTAL_AMOUNT_check           float64
roll_sum_60_TX                           float64
roll_sum_60_TX_MO                        float64
roll_sum_60_TX_check                     float64
roll_sum_60_DEFAULT                      float64
Length: 105, dtype: object

In [30]:
df['ANTIGUEDAD_d'] = (df['DATE'] - df['START_DATE']) / np.timedelta64(1, 'D')
#df['ANTIGUEDAD_w'] = (df['DATE'] - df['START_DATE']) / np.timedelta64(1, 'W')
#df['ANTIGUEDAD_m'] = (df['DATE'] - df['START_DATE']) / np.timedelta64(1, 'M')

In [31]:
df['DATE'] = df['DATE'].dt.strftime('%Y-%m-%d')
df['START_DATE'] = df['START_DATE'].dt.strftime('%Y-%m-%d')

In [19]:
df.to_csv('default_ABT2.csv', index = False)